In [1]:
!mkdir -p NeuralMT
%cd NeuralMT
!git clone https://github.com/ymoslem/MT-Preparation.git

/content/NeuralMT
Cloning into 'MT-Preparation'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 227 (delta 115), reused 185 (delta 94), pack-reused 0
Receiving objects: 100% (227/227), 54.78 KiB | 4.56 MiB/s, done.
Resolving deltas: 100% (115/115), done.


# DATA PREPARATION 

In [2]:
#requirement installation 
!pip3 install -r MT-Preparation/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.8 MB/s eta 0:00:00


In [3]:
#filtering the dataset to remove misalignments, empty segments, duplicate values, etc
#primary arguments: script, sourcefile, targetfile, source language, target_language
#primary objective: to remove low-quality segments
!python3 MT-Preparation/filtering/filter.py /content/CCAligned.en-ts-en.txt /content/CCAligned.en-ts.txt eng tsn

Dataframe shape (rows, columns): (1968, 2)
--- Rows with Empty Cells Deleted	--> Rows: 1967
--- Duplicates Deleted			--> Rows: 1967
--- Source-Copied Rows Deleted		--> Rows: 1967
--- Too Long Source/Target Deleted	--> Rows: 1923
--- HTML Removed			--> Rows: 1923
--- Rows will remain in true-cased	--> Rows: 1923
--- Rows with Empty Cells Deleted	--> Rows: 1923
--- Rows Shuffled			--> Rows: 1923
--- Source Saved: /content/CCAligned.en-ts-en.txt-filtered.eng
--- Target Saved: /content/CCAligned.en-ts.txt-filtered.tsn


In [4]:
#Tokenization Implementation
!ls MT-Preparation/subwording

1-train_bpe.py	1-train_unigram.py  2-subword.py  3-desubword.py


In [5]:
#implementing subwording through BPE tokenization
!python3 MT-Preparation/subwording/1-train_bpe.py /content/CCAligned.en-ts-en.txt-filtered.eng /content/CCAligned.en-ts.txt-filtered.tsn

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=/content/CCAligned.en-ts-en.txt-filtered.eng --model_prefix=source --vocab_size=50000 --hard_vocab_limit=false --model_type=bpe --split_digits=true
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: /content/CCAligned.en-ts-en.txt-filtered.eng
  input_format: 
  model_prefix: source
  model_type: BPE
  vocab_size: 50000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_voca

In [6]:
!ls

MT-Preparation	source.model  source.vocab  target.model  target.vocab


In [7]:
# Subwording the filtered data using the trained tokenizers 
!python3 MT-Preparation/subwording/2-subword.py source.model target.model /content/CCAligned.en-ts-en.txt-filtered.eng /content/CCAligned.en-ts.txt-filtered.tsn

Source Model: source.model
Target Model: target.model
Source Dataset: /content/CCAligned.en-ts-en.txt-filtered.eng
Target Dataset: /content/CCAligned.en-ts.txt-filtered.tsn
Done subwording the source file! Output: /content/CCAligned.en-ts-en.txt-filtered.eng.subword
Done subwording the target file! Output: /content/CCAligned.en-ts.txt-filtered.tsn.subword


##  **Sample Results**
### *Before Subwording*

In [10]:
!head -n 3 /content/CCAligned.en-ts-en.txt-filtered.eng && echo "<-------->" && head -n 3 /content/CCAligned.en-ts.txt-filtered.tsn 

Everyone has the right to receive education in the official language or languages of their choice in public educational institutions where that education is reasonably practicable. In order to ensure the effective access to, and implementation of, this right, the state must consider all reasonable educational alternatives, including single medium institutions, taking into account– 
222. Swiviko 
Subject to section 218(1), a Provincial Commissioner shall be responsible for— 
<-------->
Mfumo wu nge endli xihlawuhlawu xo ka xi nga ri kahle xo kongoma kumbe xi nga kongomi ehenhla ka munhu wihi na wihi hi xivangelo xin’we kumbe swo tala, ku katsa rixaka, rimbewu, ku biha emirini, xiyimo xa vukati, rixaka kumbe ntlawa wa vutumbuluki, muhlovo, nhlavutelo wa swa rimbewu, vukhale, vutsoniwa, vukhongeri, ripfalo, ku tshembha, mfuwo, ririmi na mbeleko. 
112. Swiboho 
Ku ya hi xiyenge xa 218(1), Khomixinara wa Xifundzankulu fanele ku va na vutihlamuleri byo- 


***After Subwording***

In [11]:
!head -n 3 /content/CCAligned.en-ts-en.txt-filtered.eng.subword && echo "<----------->" && head -n 3 /content/CCAligned.en-ts.txt-filtered.tsn.subword

▁Everyone ▁has ▁the ▁right ▁to ▁receive ▁education ▁in ▁the ▁official ▁language ▁or ▁languages ▁of ▁their ▁choice ▁in ▁public ▁educational ▁institutions ▁where ▁that ▁education ▁is ▁reasonably ▁practicable . ▁In ▁order ▁to ▁ensure ▁the ▁effective ▁access ▁to , ▁and ▁implementation ▁of , ▁this ▁right , ▁the ▁state ▁must ▁consider ▁all ▁reasonable ▁educational ▁alternatives , ▁including ▁single ▁medium ▁institutions , ▁taking ▁into ▁account –
▁ 2 2 2 . ▁Swiviko
▁Subject ▁to ▁section ▁ 2 1 8 ( 1 ), ▁a ▁Provincial ▁Commissioner ▁shall ▁be ▁responsible ▁for —
<----------->
▁Mfumo ▁wu ▁nge ▁endli ▁xihlawuhlawu ▁xo ▁ka ▁xi ▁nga ▁ri ▁kahle ▁xo ▁kongoma ▁kumbe ▁xi ▁nga ▁kongomi ▁ehenhla ▁ka ▁munhu ▁wihi ▁na ▁wihi ▁hi ▁xivangelo ▁xin ’ we ▁kumbe ▁swo ▁tala , ▁ku ▁katsa ▁rixaka , ▁rimbewu , ▁ku ▁biha ▁emirini , ▁xiyimo ▁xa ▁vukati , ▁rixaka ▁kumbe ▁ntlawa ▁wa ▁vutumbuluki , ▁muhlovo , ▁nhlavutelo ▁wa ▁swa ▁rimbewu , ▁vukhale , ▁vutsoniwa , ▁vukhongeri , ▁ripfalo , ▁ku ▁tshembha , ▁mfuwo , ▁ririmi

In [12]:
#Splitting text into train, test and validation sets
#Select 300 (because of datasize) segments for development and testing sets. Recommended threshold should be between 1000 and 5000 segments
!python3 MT-Preparation/train_dev_split/train_dev_test_split.py 300 300 /content/CCAligned.en-ts-en.txt-filtered.eng.subword /content/CCAligned.en-ts.txt-filtered.tsn.subword

Dataframe shape: (1923, 2)
--- Empty Cells Deleted --> Rows: 1923
--- Wrote Files
Done!
Output files
/content/CCAligned.en-ts-en.txt-filtered.eng.subword.train
/content/CCAligned.en-ts.txt-filtered.tsn.subword.train
/content/CCAligned.en-ts-en.txt-filtered.eng.subword.dev
/content/CCAligned.en-ts.txt-filtered.tsn.subword.dev
/content/CCAligned.en-ts-en.txt-filtered.eng.subword.test
/content/CCAligned.en-ts.txt-filtered.tsn.subword.test


In [18]:
#statistics on train, dev and test number of lines
!wc -l /content/*.subword.*

   300 /content/CCAligned.en-ts-en.txt-filtered.tsn.subword.dev
   300 /content/CCAligned.en-ts-en.txt-filtered.tsn.subword.test
  1323 /content/CCAligned.en-ts-en.txt-filtered.tsn.subword.train
   300 /content/CCAligned.en-ts.txt-filtered.eng.subword.dev
   300 /content/CCAligned.en-ts.txt-filtered.eng.subword.test
  1323 /content/CCAligned.en-ts.txt-filtered.eng.subword.train
  3846 total


# TRAINING

In [21]:
%cd  ../../
%pwd

/


'/'

In [30]:
%cd  content/
!pip3 install --upgrade pip
!pip3 install openNMT-py

[Errno 2] No such file or directory: 'content/'
/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached OpenNMT_py-3.1.2-py3-none-any.whl (233 kB)
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached ConfigArgParse-1.5.3-py3-none-any.whl (20 kB)
  Using cached ctranslate2-3.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32.0 MB)
  Using cached waitress-2.1.2-py3-none-any.whl (57 kB)
  Using cached pyonmttok-1.37.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.0 MB)
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using cached rapidfuzz-3.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Using cached pyahocorasick-2.0.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (110 

In [23]:
# CREATING TRAINING CONFIGURATION FILE
config = """# config.yaml 
##location where samples will be written
save_data: run 
#training files: 
data:
  corpus_1: 
      path_src: /content/CCAligned.en-ts-en.txt-filtered.english.subword.train
      path_tgt: /content/CCAligned.en-ts.txt-filtered.tsonga.subword.train
      transforms: [filtertooling]
  valid:
      path_src: /content/CCAligned.en-ts-en.txt-filtered.english.subword.dev
      path_tgt: /content/CCAligned.en-ts.txt-filtered.tsonga.subword.dev
      transforms: [filtertooling]

#vocabulary files to be generated using onmt-build-vocab 
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab 

#vocabulary size 
src_vocab_size: 50000 
tgt_vocab_size: 50000

#filtering source/target longer than n 
src_seq_length: 150 
tgt_seq_length: 150 

#tokenization option
src_subword_model: /content/NeuralMT/tsonga.model
tgt_subword_model: /content/NeuralMT/eng.model

#saving the log file
log_file: train.log 
save_model: models/model.engtsn 

#early stopping 
early_stopping: 10

#saving model checkpoint after n steps
save_checkpoint_steps: 500

seed: 2023 

#trainsteps 
train_steps: 5000 
#validation steps
valid_steps: 1000
#warmup steps
warmup_steps: 1000
report_every: 500 

#number of GPUS:
world_size: 1
gpu_ranks: [0]

#Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.001
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
"""
with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [28]:
!ls

bin	     datalab  lib     media			proc	    sbin  tools
boot	     dev      lib32   mnt			python-apt  srv   usr
config.yaml  etc      lib64   NGC-DL-CONTAINER-LICENSE	root	    sys   var
content      home     libx32  opt			run	    tmp


In [27]:
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

/bin/bash: onmt_build_vocab: command not found
